In [1]:
!git clone https://github.com/minhloc29/Japanese-Characters-Recognition.git

fatal: destination path 'Japanese-Characters-Recognition' already exists and is not an empty directory.


In [3]:
%cd /kaggle/working/Japanese-Characters-Recognition
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras import layers, Model
from data_loader import ClassifierDataset
from parameters import classifier_size, num_classes
from loss_and_metrics import focal_loss
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
# from sklearn.preprocessing import LabelEncoder

# data = pd.read_csv("/kaggle/input/japanese-classification/classifier.csv")
# data['image_urls'] = data['image_urls'].str.replace("data", "/kaggle/input/japanese-classification/character_images", regex = False)
# counts = data['char'].value_counts()
# valid_values = counts[counts >= 10].index
# valid_values
# filtered_rows = data[data['char'].isin(valid_values)]
# filtered_rows = filtered_rows.drop("labels", axis = 1)

# unique_chars = filtered_rows['char'].unique()
# label_encoder = LabelEncoder()
# encoded_labels = label_encoder.fit_transform(unique_chars)
# mapping =  dict(zip(unique_chars, encoded_labels))
# filtered_rows['labels'] = filtered_rows['char'].map(mapping)
# data = filtered_rows

In [12]:
image_urls = data['image_urls'].to_list()

labels = data['labels'].to_list()
train_img_urls, test_img_urls, train_labels, test_labels = train_test_split(
    image_urls, labels, test_size=0.2, random_state=42, stratify=labels
)
train_classifier = ClassifierDataset(image_urls=train_img_urls, labels=train_labels, batch_size = 64, img_size = (64, 64), augment=True, shuffle=True)
test_classifier = ClassifierDataset(image_urls=test_img_urls, labels=test_labels, batch_size = 64,img_size = (64, 64), augment=False, shuffle=False)
train = train_classifier.create_tf_dataset()
test = test_classifier.create_tf_dataset()

In [7]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
from tensorflow.keras import mixed_precision

# Set the mixed precision policy to 'mixed_bfloat16'
policy = mixed_precision.Policy('mixed_bfloat16')
mixed_precision.set_global_policy(policy)

Num GPUs Available:  2


In [8]:
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

csv = CSVLogger(
    filename = "/kaggle/working/Japanese-Characters-Recognition/training_log.csv", append = True
)
early = EarlyStopping(
    monitor = 'val_loss', patience = 2
)
checkpoint = ModelCheckpoint(
    filepath = "model.keras",
    monitor = 'val_loss',
    verbose = 1,
    mode = 'min',
    save_best_only = True,
    save_weights_only = False,
    save_freq = 'epoch',
    initial_value_threshold = 0.9
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2)

In [18]:
batch_size = 64
len_train = int(len(train_img_urls) / batch_size)
len_test = int(len(test_img_urls) / batch_size)

strategy = tf.distribute.MirroredStrategy()
tf.debugging.set_log_device_placement(True)

with strategy.scope():
    model = tf.keras.models.load_model("/kaggle/working/Japanese-Characters-Recognition/first.keras")
    optim = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss=my_loss, optimizer=optim, metrics=["accuracy"])

history = model.fit(train, epochs=3, validation_data=test, 
                   steps_per_epoch = len_train, validation_steps= len_test, callbacks = [csv, early, checkpoint, reduce_lr])


Epoch 1/3


2024-12-16 11:20:34.261691: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/cond/else/_744/cond/StatefulPartitionedCall/functional_3_1/dropout_5_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
/tmp/ipykernel_23/1443643313.py:26: UserWarning: Argument 'alpha_affine' is not valid and will be ignored.
  A.ElasticTransform(alpha=1, sigma=50, alpha_affine=30, p=0.3),  # Distortions for handwriting


8458/8458 ━━━━━━━━━━━━━━━━━━━━ 0s 456ms/step - accuracy: 0.6841 - loss: 0.3568
Epoch 1: val_loss improved from 0.43422 to 0.10215, saving model to model.keras
8458/8458 ━━━━━━━━━━━━━━━━━━━━ 4781s 560ms/step - accuracy: 0.6841 - loss: 0.3568 - val_accuracy: 0.8748 - val_loss: 0.1022 - learning_rate: 0.0010
Epoch 2/3
8458/8458 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step - accuracy: 0.8735 - loss: 0.0969
Epoch 2: val_loss improved from 0.10215 to 0.05897, saving model to model.keras
8458/8458 ━━━━━━━━━━━━━━━━━━━━ 4763s 563ms/step - accuracy: 0.8735 - loss: 0.0969 - val_accuracy: 0.9145 - val_loss: 0.0590 - learning_rate: 0.0010
Epoch 3/3
8327/8458 ━━━━━━━━━━━━━━━━━━━━ 59s 458ms/step - accuracy: 0.9177 - loss: 0.0528 
Epoch 3: val_loss improved from 0.05897 to 0.04249, saving model to model.keras
8458/8458 ━━━━━━━━━━━━━━━━━━━━ 4783s 566ms/step - accuracy: 0.9178 - loss: 0.0527 - val_accuracy: 0.9356 - val_loss: 0.0425 - learning_rate: 0.0010


In [20]:
model.save("first.keras")